### 1. 升级boto3, sagemaker python sdk

In [ ]:
!pip install --upgrade boto3 sagemaker

In [ ]:
import re
import os
import json
import uuid

import numpy as np
import pandas as pd
from time import gmtime, strftime


import boto3
import sagemaker

from sagemaker import get_execution_role,session

role = get_execution_role()

sage_session = session.Session()
bucket = sage_session.default_bucket()
aws_region = boto3.Session().region_name
account_id = sage_session.account_id()
model = 'cosyvoice'

print(f'sagemaker sdk version: {sagemaker.__version__}\nrole:  {role}  \nbucket:  {bucket}')

In [ ]:
!sed -i "s/BUCKET=/BUCKET='{bucket}'/" code/inference.py

In [ ]:
!sed -i "s/S3_Prefix=/S3_Prefix='{model}\/asyncinvoke\/out'/" code/inference.py

### 2. 编译docker image

**根据不同的需求，选择不同的模型进行部署**

`!sh build_docker.sh $model_name`

- 预训练音色 模式

In [ ]:
model_name='CosyVoice-300M-SFT'

In [ ]:
!sh build_docker.sh {model_name}

- 复刻音色模式（同语种&跨语种）

In [ ]:
model_name='CosyVoice-300M'

In [ ]:
!sh build_docker.sh {model_name}

- 高级角色音色(给定角色system_prompt描述)

In [ ]:
model_name='CosyVoice-300M-Instruct'

In [ ]:
!sh build_docker.sh {model_name}

### 3. 部署AIGC推理服务

#### 3.1 创建dummy model_data 文件(真正的模型使用code/infernece.py进行加载)

In [ ]:
model=model_name.lower()

In [ ]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, model)
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, model)
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

#### 3.2 创建 model 配置

In [ ]:
boto3_session = boto3.session.Session()
current_region=boto3_session.region_name

client = boto3.client("sts")
account_id=client.get_caller_identity()["Account"]

client = boto3.client('sagemaker')

#使用步骤2编译好的docker images
container = f'{account_id}.dkr.ecr.{current_region}.amazonaws.com/{model}'
model_data = f's3://{bucket}/{model}/assets/model.tar.gz'

model_name = f'{account_id}-{model}'
role = get_execution_role()

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data,
    'Environment':{
        's3_bucket': bucket,
        'model_name': model_name #默认为runwayml/stable-diffusion-v1-5
    }
}
create_model_response = client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer=primary_container
)

In [ ]:
time_tag = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
variant_name = f'variant-{model_name}-{time_tag}'
endpoint_config_name = f'config-{model_name}-{time_tag}'

response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': variant_name,
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.g4dn.2xlarge',
            'InitialVariantWeight': 1
        },
    ],
    AsyncInferenceConfig={
        'OutputConfig': {
            'S3OutputPath': f's3://{bucket}/{model}/asyncinvoke/out/'
        }
    }
)

In [ ]:
time_tag = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
variant_name = f'variant-{model_name}-{time_tag}'
endpoint_config_name = f'config-{model_name}-{time_tag}'

response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': variant_name,
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.g4dn.2xlarge',
            'InitialVariantWeight': 1
        },
    ]
)

In [ ]:
endpoint_name = f'{model_name}-endpoint'

response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

print(f'终端节点:{endpoint_name} 正在创建中，首次启动中会加载模型，请耐心等待, 请在控制台上查看状态')

### 4. 测试

In [ ]:
def tts_by_sm_endpoint(data, sm_client, endpoint_name):
    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(data),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    return json_obj

- 预制音色推理

role的可选项为['中文女', '中文男', '日语男', '粤语女', '英文女', '英文男', '韩语女']

In [ ]:
runtime_client = boto3.client('runtime.sagemaker')
data = {
    "tts_text": '你好，我是GenAI专家，你的速度快不快',
    "role" : "中文女"
}
tts_by_sm_endpoint(data, runtime_client, endpoint_name)

- 模仿音色推理

**同语言**

In [ ]:
data = {
    "tts_text": "收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。",
    "prompt_text" : "希望你以后能够做的比我还好呦。",
    "prompt_audio" : "https://github.com/FunAudioLLM/CosyVoice/raw/main/zero_shot_prompt.wav"
}
tts_by_sm_endpoint(data, runtime_client, endpoint_name)

**跨语言**

*zero_shot usage, <|zh|><|en|><|jp|><|yue|><|ko|> for Chinese/English/Japanese/Cantonese/Korean*

In [ ]:
data = {
    "tts_text": "<|yue|>对唔住，有钱真系可以为所欲为",
    "prompt_audio" : "https://github.com/FunAudioLLM/CosyVoice/raw/main/cross_lingual_prompt.wav"
}
tts_by_sm_endpoint(data, runtime_client, endpoint_name)

- 高级角色音色推理

In [ ]:
data = {
    "tts_text": "在面对挑战时，他展现了非凡的<strong>勇气</strong>与<strong>智慧</strong>。",
    "role" : "中文男",
    "instruct_text" : "Theo \'Crimson\', is a fiery, passionate rebel leader. Fights with fervor for justice, but struggles with impulsiveness."
}

tts_by_sm_endpoint(data, runtime_client, endpoint_name)